In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q transformers scikit-learn datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import json
import re

# 🔢 Preprocesamiento y métricas
from sklearn.preprocessing import LabelEncoder
import evaluate

# 📚 Carga de datasets
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset


# Modelos y entrenamiento con Hugging Face
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    T5Config,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback,
)

# 🔥 PyTorch
import torch
from torch.utils.data import DataLoader

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np

# Cargar los tres modelos y tokenizadores
modelo_bert_path = "/content/drive/MyDrive/TFG/models/finetuned_bert-base-uncased_loss/"
modelo_roberta_path = "/content/drive/MyDrive/TFG/models/finetuned_roberta-base_loss/"
modelo_t5_path = "/content/drive/MyDrive/TFG/models/finetuned_t5-small_eval_loss/"

modelo_bert = AutoModelForSequenceClassification.from_pretrained(modelo_bert_path)
tokenizer_bert = AutoTokenizer.from_pretrained(modelo_bert_path)

modelo_roberta = AutoModelForSequenceClassification.from_pretrained(modelo_roberta_path)
tokenizer_roberta = AutoTokenizer.from_pretrained(modelo_roberta_path)

modelo_t5 = AutoModelForSequenceClassification.from_pretrained(modelo_t5_path)
tokenizer_t5 = AutoTokenizer.from_pretrained(modelo_t5_path)

# Frases de prueba
frases_prueba = [
    "Add this song to my playlist",  # ADD
    "Could you remove this track?",  # REMOVE
    "Show me my current playlist",  # VIEW
    "Clear the entire playlist",    # CLEAR
    "Add this song to my playlist",  # ADD
    "Remove this track from my list",  # REMOVE
    "Show me my current playlist",  # VIEW
    "Clear my entire playlist",  # CLEAR
    "Could you please delete all the songs?",  # CLEAR
    "Can you add a song to my playlist?",  # ADD
    "I'd like to see what songs I have added",  # VIEW
    "Take this song out of my list",  # REMOVE
    "Hey, can u clear my playlist",     # CLEAR
    "can u delete the song Crazy",   # REMOVE
]

# Tokenizar las frases con los tres tokenizadores
inputs_bert = tokenizer_bert(frases_prueba, padding=True, truncation=True, return_tensors="pt")
inputs_roberta = tokenizer_roberta(frases_prueba, padding=True, truncation=True, return_tensors="pt")
inputs_t5 = tokenizer_t5(frases_prueba, padding=True, truncation=True, return_tensors="pt")

# Obtener las predicciones de los tres modelos
def get_predictions(model, inputs):
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        return torch.argmax(outputs.logits, dim=1).numpy()

predicciones_bert = get_predictions(modelo_bert, inputs_bert)
predicciones_roberta = get_predictions(modelo_roberta, inputs_roberta)
predicciones_t5 = get_predictions(modelo_t5, inputs_t5)

# Promediar las predicciones de los tres modelos
# En lugar de tomar la predicción de cada modelo por separado, tomamos el promedio de las predicciones
# para cada clase y luego asignamos la clase con mayor puntuación.
predicciones_ensemble = np.argmax(np.stack([predicciones_bert, predicciones_roberta, predicciones_t5], axis=0), axis=0)

# Etiquetas correspondientes a las predicciones
etiquetas = ["add", "clear", "remove", "view"]

# Mostrar los resultados
for frase, pred in zip(frases_prueba, predicciones_ensemble):
    print(f"Frase: '{frase}' → Predicción: {etiquetas[pred]}")



Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/TFG/models/finetuned_t5-small_eval_loss/ and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Frase: 'Add this song to my playlist' → Predicción: add
Frase: 'Could you remove this track?' → Predicción: add
Frase: 'Show me my current playlist' → Predicción: add
Frase: 'Clear the entire playlist' → Predicción: add
Frase: 'Add this song to my playlist' → Predicción: add
Frase: 'Remove this track from my list' → Predicción: add
Frase: 'Show me my current playlist' → Predicción: add
Frase: 'Clear my entire playlist' → Predicción: add
Frase: 'Could you please delete all the songs?' → Predicción: add
Frase: 'Can you add a song to my playlist?' → Predicción: clear
Frase: 'I'd like to see what songs I have added' → Predicción: add
Frase: 'Take this song out of my list' → Predicción: add
Frase: 'Hey, can u clear my playlist' → Predicción: add
Frase: 'can u delete the song Crazy' → Predicción: add


In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Cargar el modelo y el tokenizador desde el directorio guardado
model_path = "/content/drive/MyDrive/TFG/models/finetuned_bert-base-uncased_loss"
modelo = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Definir frases de prueba
frases_prueba = [
    "Add this song to my playlist",  # ADD
    "Could you remove this track?",  # REMOVE
    "Show me my current playlist",  # VIEW
    "Clear the entire playlist",    # CLEAR
    "Add this song to my playlist",  # ADD
    "Remove this track from my list",  # REMOVE
    "Show me my current playlist",  # VIEW
    "Clear my entire playlist",  # CLEAR
    "Could you please delete all the songs?",  # CLEAR
    "Can you add a song to my playlist?",  # ADD
    "I'd like to see what songs I have added",  # VIEW
    "Take this song out of my list",  # REMOVE
    "Hey, can u clear my playlist",     # CLEAR
    "can u delete the song Crazy",   # REMOVE
]

# Tokenizar las frases de prueba
inputs = tokenizer(frases_prueba, padding=True, truncation=True, return_tensors="pt")

# Pasar las frases al modelo para obtener predicciones
modelo.eval()  # Poner el modelo en modo evaluación
with torch.no_grad():
    outputs = modelo(**inputs)

# Convertir los logits a predicciones
predicciones = torch.argmax(outputs.logits, dim=1).numpy()

# Mapear los índices a las etiquetas
etiquetas = ["add", "clear", "remove", "view"]

# Mostrar las predicciones
for frase, pred in zip(frases_prueba, predicciones):
    print(f"Frase: '{frase}' → Predicción: {etiquetas[pred]}")


Frase: 'Add this song to my playlist' → Predicción: add
Frase: 'Could you remove this track?' → Predicción: remove
Frase: 'Show me my current playlist' → Predicción: view
Frase: 'Clear the entire playlist' → Predicción: clear
Frase: 'Add this song to my playlist' → Predicción: add
Frase: 'Remove this track from my list' → Predicción: remove
Frase: 'Show me my current playlist' → Predicción: view
Frase: 'Clear my entire playlist' → Predicción: clear
Frase: 'Could you please delete all the songs?' → Predicción: clear
Frase: 'Can you add a song to my playlist?' → Predicción: add
Frase: 'I'd like to see what songs I have added' → Predicción: view
Frase: 'Take this song out of my list' → Predicción: remove
Frase: 'Hey, can u clear my playlist' → Predicción: clear
Frase: 'can u delete the song Crazy' → Predicción: remove


In [ ]:
# Cargar el modelo y el tokenizador
model_path = "/content/drive/MyDrive/TFG/models/finetuned_t5-small_eval_loss"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5TokenizerFast.from_pretrained(model_path)

# Lista de frases de prueba
frases_prueba = [
    "Add this song to my playlist",  # ADD
    "Could you remove this track?",  # REMOVE
    "Show me my current playlist",  # VIEW
    "Clear the entire playlist",    # CLEAR
    "Remove this track from my list",  # REMOVE
    "Show me my current playlist",  # VIEW
    "Clear my entire playlist",  # CLEAR
    "Could you please delete all the songs?",  # CLEAR
    "Can you add a song to my playlist?",  # ADD
    "I'd like to see what songs I have added",  # VIEW
    "Take this song out of my list",  # REMOVE
    "Hey, can u clear my playlist",     # CLEAR
    "can u delete the song Crazy",   # REMOVE
]

# Preprocesar y generar predicciones para cada frase
for frase in frases_prueba:
    # Preprocesar la entrada
    input_ids = tokenizer("Clasifica: " + frase, return_tensors="pt").input_ids
    attention_mask = tokenizer("Clasifica: " + frase, return_tensors="pt").attention_mask

    # Generar la predicción
    model.eval()
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=10)

    # Decodificar la predicción
    decoded_pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Imprimir la predicción
    print(f"Frase: {frase}")
    print(f"Predicción: {decoded_pred}")
    print("-" * 50)


Frase: Add this song to my playlist
Predicción: add
--------------------------------------------------
Frase: Could you remove this track?
Predicción: remove
--------------------------------------------------
Frase: Show me my current playlist
Predicción: view
--------------------------------------------------
Frase: Clear the entire playlist
Predicción: clear
--------------------------------------------------
Frase: Remove this track from my list
Predicción: clear
--------------------------------------------------
Frase: Show me my current playlist
Predicción: view
--------------------------------------------------
Frase: Clear my entire playlist
Predicción: clear
--------------------------------------------------
Frase: Could you please delete all the songs?
Predicción: clear
--------------------------------------------------
Frase: Can you add a song to my playlist?
Predicción: add
--------------------------------------------------
Frase: I'd like to see what songs I have added
Pred

In [ ]:
# Cargar el modelo y el tokenizador desde el directorio guardado
model_path = "/content/drive/MyDrive/TFG/models/finetuned_t5-small_eval_loss"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5TokenizerFast.from_pretrained(model_path)

input_text = "can u delete the song Crazy"

# Preprocesar la entrada añadiendo el prefijo necesario
input_ids = tokenizer("Clasifica: " + input_text, return_tensors="pt").input_ids
attention_mask = tokenizer("Clasifica: " + input_text, return_tensors="pt").attention_mask

# Generar la predicción
model.eval()  # Asegurarse de que el modelo esté en modo de evaluación
with torch.no_grad():  # No necesitamos gradientes para la inferencia
    generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=10)

# Decodificar la predicción
decoded_pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(f"Predicción: {decoded_pred}")




Predicción: remove
